# Computational Social Science Project #2 

*Group number:* 5

*Group members:* Alonzo Ackerman, Christopher Crawford, Annie Helms    

*Semester:* Fall 2021


Below we fill in some of the code you might use to answer some of the questions. Here are some additional resources for when you get stuck:
* Code and documentation provided in the course notebooks  
* [Markdown cheatsheet](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet) to help with formatting the Jupyter notebook
* Try Googling any errors you get and consult Stack Overflow, etc. Someone has probably had your question before!
* Send KQ a pull request on GitHub flagging the syntax that's tripping you up 

## 1. Introduction/Setup

#### a) Import relevant libraries
Add the other libraries you need for your code below and/or as you go. 

In [ ]:
# import libraries you might need here 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# use random seed for consistent results 
np.random.seed(273)

#### b) Read in and inspect data frame 
Read in the data frame and look at some of its attributes. 

In [ ]:
diabetes = pd.read_csv('Diabetes with Population Info by County 2017.csv', delimiter = ',' , 
                
                       #CountyFips needs to be a string so leading 0 isn't dropped (this is only if you want to make choropleth map): 
                       dtype={"CountyFIPS": str}) 


In [ ]:
# look at the dimensions of the diabetes data frame
print('shape: ', diabetes.size) 

In [ ]:
pd.set_option('display.max_rows', 100) # tells pandas how many rows to display when printing so results don't get truncated

# look at the data types for each column in diabetes df 
print('data types:', diabetes.dtypes)

Immediately, we see that some of the features that should be numeric (e.g., Diabetes_Number, Obesity_Number,  and Physical_Inactivity_Number) are not. We can check to see what the non-numeric values are in a column where we are expecting numeric information with a combination of `str.isnumeric()` and `unique()`.

In [ ]:
# Return rows where the column "Diabetes_Number" is non-numeric and get the unique values of these rows
# the "~" below in front of diabetes negates the str.isnumeric() so it only takes non-numeric values
print(diabetes[~diabetes["Diabetes_Number"].str.isnumeric()]["Diabetes_Number"].unique()) 

In [ ]:
# Now do the same as above, but for "Obesity_Number" :
print(diabetes[~diabetes["Obesity_Number"].str.isnumeric()]["Obesity_Number"].unique())


The values contained in the two columns above making them objects (rather than integers) appear to be strings like "No Data" and "Suppressed." Let's drop those rows in the next section, and also recode Physical_Inactivity_Number to be an integer. 

#### c. Recode variables

Convert 'Diabetes_Number', 'Obesity_Number', and 'Physical_Inactivity_Number' to integers below so we can use them in our analysis. Also fill in the object type we want to recode 'sex and age_total population_65 years and over_sex ratio (males per 100 females)' to. 

In [ ]:
# Diabetes
# keep only useful info about our target feature, i.e., where diabetes_number not = 'Suppressed'
diabetes = diabetes[diabetes['Diabetes_Number']!="Suppressed"]  # note that the inside reference to the diabetes df identifies the column, and the outer calls specific rows according to a condition 


# use the astype method on Diabetes_Number to convert it to an integer...if you are not sure, what does the astype() documentation tell you are possible arguments? 
diabetes['Diabetes_Number'] = diabetes['Diabetes_Number'].astype('int64', copy=False) 

# Obesity 
diabetes = diabetes[diabetes['Obesity_Number']!="No Data"]
diabetes['Obesity_Number'] = diabetes['Obesity_Number'].astype('int64', copy=False)


# Physical Inactivity
diabetes['Physical_Inactivity_Number'] = diabetes['Physical_Inactivity_Number'].astype('int64', copy=False)

# 65+ sex ratio had one "-" in it so let's drop that row first
diabetes = diabetes[diabetes['sex and age_total population_65 years and over_sex ratio (males per 100 females)']!= "-"]
# change to numeric (specifically, integer or float?) from string (because originally included the "-" )
diabetes['sex and age_total population_65 years and over_sex ratio (males per 100 females)'] = diabetes['sex and age_total population_65 years and over_sex ratio (males per 100 females)'].astype('float64', copy=True)


We should probably scale our count variables to be proportional to county population. We create the list 'rc_cols' to select all the features we want to rescale, and then use the `.div()` method to avoid typing out every single column we want to recode. 

In [ ]:
# select count variables to rc to percentages; make sure we leave out ratios and our population variable b/c these don't make sense to scale by population
rc_cols = [col for col in diabetes.columns if col not in ['County', 'State', 'CountyFIPS', 
                                                        'sex and age_total population_65 years and over_sex ratio (males per 100 females)', 'sex and age_total population_sex ratio (males per 100 females)', 'sex and age_total population_18 years and over_sex ratio (males per 100 females)',  
                                                        'race_total population']]
           
diabetes[rc_cols] = diabetes[rc_cols].apply(pd.to_numeric, errors='coerce') # recode all selected columns to numeric

# divide all columns but those listed above by total population to calculate rates
diabetes[rc_cols] = diabetes[rc_cols].div(diabetes['race_total population'], axis=0)

Let's check our work. Are all rates bounded by 0 and 1 as expected? 

In [ ]:
pd.set_option('display.max_columns', None)
# inspect recoded values
diabetes_summary = diabetes.describe().transpose() # note we use the transpose method rather than .T because this object is not a numpy array
  
# check recoding 
with pd.option_context('display.max_rows', 100, 'display.max_columns', None): 
    display(diabetes_summary.iloc[ : ,[0,1,3,7]]) # select which columns in the summary table we want to present

#### d. Check for duplicate columns

There are a lot of columns in this data frame. Let's see if there are any are duplicates. 

In [ ]:
# I used Google to figure this out, and adapted this example for our purposes:  
# source: https://thispointer.com/how-to-find-drop-duplicate-columns-in-a-dataframe-python-pandas/ 
def getDuplicateColumns(df):
    '''
    Get a list of duplicate columns.
    It will iterate over all the columns in dataframe and find the columns whose contents are duplicate.
    :param df: Dataframe object
    :return: List of columns whose contents are duplicates.
    '''
    duplicateColumnNames = set()
    # Iterate over all the columns in dataframe
    for x in range(df.shape[1]):
        # Select column at xth index.
        col = df.iloc[:, x]
        # Iterate over all the columns in DataFrame from (x+1)th index till end
        for y in range(x + 1, df.shape[1]):
            # Select column at yth index.
            otherCol = df.iloc[:, y]
            # Check if two columns at x 7 y index are equal
            if col.equals(otherCol):
                duplicateColumnNames.add(df.columns.values[y])
    return list(duplicateColumnNames)

duplicateColumnNames = list(getDuplicateColumns(diabetes))
print('Duplicate Columns are as follows: ')
duplicateColumnNames

In [ ]:
# now drop list of duplicate features from our df using the .drop() method
diabetes = diabetes.drop(columns=(duplicateColumnNames))

## 2. Exploratory Data Analysis

In [ ]:
# insert your EDAs and interpretations in this section 

### Annie's Exploratory Data Analysis:

I will create a chloropleth map of the USA with scaled diabetes number across each county. To create this map, the following libraries should be installed and imported.

In [ ]:
# uncomment to install required packages
#!pip install -U plotly
#!pip install -U dash
#!pip install -U kaleido

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
    
# This file contains geometry information for US counties, with corresponding FIPS code as column 'id'
counties["features"][0]

In [ ]:
# import packages for plotting
import kaleido
import plotly.express as px
import plotly.io as pio

In [ ]:
fig = px.choropleth(diabetes, geojson=counties, locations='CountyFIPS', color='Diabetes_Number',
                           color_continuous_scale="Viridis",
                           range_color=(0, 0.2),
                           scope="usa",
                           labels={'Diabetes_Number':'Rate of Diabetes',
                                  'CountyFIPS': 'County ID'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

# fig takes too long to render, so written to new file which will open when the cell is run
# note, as this file 'saves' each time the cell is run, the file name has been added to a .gitignore file
fig.write_html("eda_annie_plot.html", auto_open=True)

Based on the chloropleth figure, there appears to be a relationship between geography and the diabetes rate. In general, there are higher rates of diabetes in the southeastern states of the country, including Louisiana, Mississippi, Alabama, Georgia, South Carolina, and North Carolina. The figure suggests that community-wide factors, such as culture and social networks, may have an influence upon the rate of diabetes in addition to individual-level factors, such as age and physical inactivity.

### Chris's Exploratory Data Analysis:

In [ ]:
plt.figure(figsize=(10, 8)) #set figure size

plt.scatter(diabetes["Physical_Inactivity_Number"], 
            diabetes["Diabetes_Number"],
            marker = "o",
            alpha = 0.25,
            c = diabetes["sex and age_total population_65 years and over"], #change colors
            cmap = "hot")
plt.xlabel("Physical Inactivity", fontsize = 16)
plt.ylabel("Diabetes", fontsize = 16)
plt.colorbar().set_label("Over 65", fontsize=16)

As shown in the scatterplot above, there is a strong relationship between physical inactivity and diabetes, with higher levels of physical inactivity being associated with greater diabetes. The color of each observation represents the percentage of the population over the age of 65, and it appears that counties with younger populations (darker colors) have lower levels of diabetes and lower levels of physical inactivity. 

## 3. Prepare to Fit Models

### 3.1 Finalize Data Set

We've already cleaned up the data, but we can make a few more adjustments before partitioning the data and training models. Let's recode 'State' to be a categorical variable using `pd.get_dummies` and drop 'County' using `.drop()` because 'CountyFIPS' is already a unique identifier for the county. 

In [ ]:
# create dummy features out of 'State' , which might be related to diabetes rates 
diabetes_clean = pd.get_dummies(diabetes, 
                               columns = ['State'],  
                               drop_first = True) # only create 49 dummies by dropping first in category

# drop 'County' variable
diabetes_clean = diabetes_clean.drop(labels = ['County'],
                               axis = 1) # which axis tells python we want to drop columns rather than index rows?

# look at first 10 rows of new data frame 
diabetes_clean.head(10)

### 3.2/3.3 Partition Data and Feature Selection

Now, we will partition our data to prepare it for the training process. We will use 60% train—20% validation—20% test in this case. More data in the training set lowers bias, but then increases variance in the validation/test sets. Balancing between bias and variance with choice of these set sizes is important as we want to ensure that there is enough data to train on to get good predictions, but also want to make sure our hold-out sets are representative enough.

In [ ]:
from sklearn.model_selection import train_test_split

# Set y 
y = diabetes_clean['Diabetes_Number']

# X (everything except diabetes, our target)
X = diabetes_clean.drop(["Diabetes_Number"], axis = 1)

We should also preprocess our data. Using the `preprocessing` module from sklearn, let's scale our features so that they are mean-centered.

In [ ]:
from sklearn import preprocessing

X = preprocessing.scale(X)

We can also get rid of the 0 variance features using the `VarianceThreshold()` method from `feature_selection`. 

In [ ]:
from sklearn import feature_selection

selector = feature_selection.VarianceThreshold(0)
X = selector.fit_transform(X)

And finally, let's split our data:

In [ ]:
# split the data
# train_test_split returns 4 values: X_train, X_test, y_train, y_test, so how do we create a 60-20-20 train-validate-test split? 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

X_train, X_validate, y_train, y_validate = train_test_split(X_train, y_train, test_size = 0.25)

## 4. Train Models

In [ ]:
# train your three models in this section 

### Model 1: Linear Regression (OLS)

- Model Explanation and Logic

Linear regression algorithms can take an input of a set of predictor variables and predict the value of a continous output variable. OLS is fairly interpretable, where a coefficient is fitted to each predictor value, in a sense assigning the weight of the variable to the outcome. For the current data set, our outcome variable, `Diabetes_Number`, is a continuous value. Therefore, OLS is a logical choice for a model. However, OLS does not constraint the model coefficients in any way, or drop out variables that are not correlated to the outcome variable. As there are many predictor variables, the regression model will not be as interpretable as it could be if we were to use Ridge or LASSO regression.

- Train Model

The OLS model will be trained using the training set and hyperparameters will be tuned using `GridSearchCV`.

In [ ]:
# import necessary libraries
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.model_selection import GridSearchCV

Define function to calculate the mean squared error:

In [ ]:
def rmse(pred, actual):
    return np.sqrt(np.mean((pred - actual) ** 2))

In [ ]:
# create a model
lin_reg = LinearRegression()

# tune hyperparameters
param_grid = {'fit_intercept': ['True', 'False'],
              'normalize': ['True', 'False']}

lin_grid_reg = GridSearchCV(lin_reg, param_grid, cv=3)
lin_grid_reg.fit(X_train, y_train)

best_index = np.argmax(lin_grid_reg.cv_results_["mean_test_score"])
best_lin_pred = lin_grid_reg.best_estimator_.predict(X_validate)

print(lin_grid_reg.cv_results_["params"][best_index])
print('Best CV R^2:', max(lin_grid_reg.cv_results_["mean_test_score"]))
print('Validation R^2:', lin_grid_reg.score(X_validate, y_validate))
print('Validation RMSE', rmse(best_lin_pred, y_validate))

In [ ]:
# recreate model, using selected hyperparameters
lin_reg = LinearRegression(fit_intercept=True, normalize=True)
lin_model = lin_reg.fit(X_train, y_train)

Obtain the coefficients for each predictor variable, and the intercept of the regression line:

In [ ]:
print(lin_model.coef_)
print(lin_model.intercept_)

Obtain highest 10 (absolute value) coefficients:

In [ ]:
# Create a dataframe with the coefficient and feature names
X_asdf = diabetes_clean.drop(["Diabetes_Number"], axis=1)
lin_reg_data = pd.DataFrame([lin_model.coef_, X_asdf.columns]).T
lin_reg_data.columns = ['Coefficient', 'Feature']
lin_reg_data = lin_reg_data.convert_dtypes()

In [ ]:
# resort dataframe and show highest 15 absolute value rows
pd.set_option('max_colwidth', 400)
lin_reg_data_sorted = lin_reg_data.reindex(lin_reg_data.Coefficient.abs().sort_values(ascending=False).index)
lin_reg_data_sorted.head(15)

Calculate root means squared error to evaluate how well the model fits the training data (what is the difference between the predicted values and the training values)

In [ ]:
# predict the rate of diabetes
lin_pred = lin_model.predict(X_train)
rmse(lin_pred, y_train)

In [ ]:
# now a visual examination

# plot the residuals on a scatter plot
plt.scatter(y_train, lin_pred)
plt.title('Linear Model (OLS) Predicted v. Actual')
plt.xlabel('actual value')
plt.ylabel('predicted value')
plt.show()

If we imagine a best fit line for the plot above, the slope of the line would be pretty close to 1, meaning that the OLS model is a very good predictor for the training data. 

### Model 2: Ridge Regression

Like OLS, ridge regression estimates parameters that minimize the sum of the squared residuals; however, unlike OLS, ridge regression also seeks to minimize the sum of the squared coefficients, as a function of a tuning parameter. The rationale behind ridge regression is that by shrinking the estimates toward zero, the variance of the estimated model is reduced. Moreover, ridge regression is particularly well suited to models consisting of a larger number of predictors.\

The value of tuning parameter will be selected using the `GridSearceCV()` method. Values between 0.1 and 1 (in increments of 0.1) will be considered.

In [ ]:
# RIDGE: 

#initialize model
ridge_reg = Ridge() 

param_grid = {'alpha': np.arange(.1, 1, .1),
              'normalize': ['True', 'False'],
              'fit_intercept': ['True', 'False'],
              'solver': ['auto', 'svd', 'cholesky', 'lsqr']}

ridge_grid_reg = GridSearchCV(ridge_reg, param_grid, cv=3)
ridge_grid_reg.fit(X_train, y_train)

best_index = np.argmax(ridge_grid_reg.cv_results_["mean_test_score"])
best_ridge_pred = ridge_grid_reg.best_estimator_.predict(X_validate)

print(ridge_grid_reg.cv_results_["params"][best_index])
print('Best CV R^2:', max(ridge_grid_reg.cv_results_["mean_test_score"]))
print('Validation R^2:', ridge_grid_reg.score(X_validate, y_validate))
print('Validation RMSE', rmse(best_ridge_pred, y_validate))

The hyperparameters for the best fitting model are shown above. The tuning parameter, denoted as alpha, for the best fitting model has a value of 0.9. The validation RMSE is 0.020, which is slightly better than the OLS validation RMSE of 0.021. The best fitting ridge model is also explaining more validation variance (50%) than the OLS model (45%).

In [ ]:
# recreate model, using selected hyperparameters
ridge_reg = Ridge(alpha=0.9, fit_intercept= True, normalize= True, solver= 'svd')
ridge_model = ridge_reg.fit(X_train, y_train)

In [ ]:
print(ridge_model.coef_)
print(ridge_model.intercept_)

In [ ]:
# Create a dataframe with the coefficient and feature names
X_names = diabetes_clean.drop(["Diabetes_Number"], axis=1)
ridge_reg_params = pd.DataFrame([ridge_model.coef_, X_names.columns]).T
ridge_reg_params.columns = ['Coefficient', 'Feature']
ridge_reg_params = ridge_reg_params.convert_dtypes()

In [ ]:
# sort dataframe and show highest 15 absolute value rows
pd.set_option('max_colwidth', 400)
ridge_reg_params_sorted = ridge_reg_params.reindex(ridge_reg_params.Coefficient.abs().sort_values(ascending=False).index)
ridge_sorted = ridge_reg_params_sorted.head(15)
ridge_sorted

In [ ]:
# Plot coefficients
ax = sns.barplot(x = "Coefficient", y = "Feature", data = ridge_sorted)
ax.set_title("Top 15 Ridge Coefficients")
plt.show()

As shown in the table and barplot above, the coefficients for `Physical_Inactivity_Numer` and `Obestity_Number` are the largest, suggesting that those two variables are the best predictors of `Diabetes_Number`.

The ridge RMSE for the training data is listed below, and is slightly larger than the OLS RMSE (0.020 vs. 0.019)

In [ ]:
# predict the rate of diabetes
ridge_pred = ridge_model.predict(X_train)
rmse(ridge_pred, y_train)

In [ ]:
# plot actual vs predicted values
plt.scatter(y_train, ridge_pred)
plt.title('Ridge Regression Predicted v. Actual')
plt.xlabel('actual value')
plt.ylabel('predicted value')
plt.show()

As shown in the scatterplot above, the actual values for `Diabetes_Number` in the training data and the values predicted by the ridge model are strongly correlated.

### Model 3: LASSO Regression

Like ridge regression, the LASSO estimates paramters that minimize both the sum of the squared residuals and a shrinkage penalty, but whereas the penalty for ridge regression is the sum of the squared coeffiecent estimates, the penalty for the LASSO is the sum of the absolute values of the coefficient estimates. This has the desirable property of shrinking some coefficent estimates to zero when the tuning paramter is sufficiently large. Thus, in addition to improving upon OLS by reducing the variance of the estimated model, the LASSO also performs feature selection by forcing some estimates to be zero.

In [ ]:
# LASSO: 

#initialize model
lasso_reg = Lasso(max_iter=10000) 

param_grid = {'alpha': np.arange(.01, 1, .01),
              'normalize': ['True', 'False'],
              'fit_intercept': ['True', 'False'],
              'selection': ['cyclic', 'random']}

lasso_grid_reg = GridSearchCV(lasso_reg, param_grid, cv=3)
lasso_grid_reg.fit(X_train, y_train)

best_index = np.argmax(lasso_grid_reg.cv_results_["mean_test_score"])
best_lasso_pred = lasso_grid_reg.best_estimator_.predict(X_validate)

print(lasso_grid_reg.cv_results_["params"][best_index])
print('Best CV R^2:', max(lasso_grid_reg.cv_results_["mean_test_score"]))
print('Validation R^2:', lasso_grid_reg.score(X_validate, y_validate))
print('Validation RMSE', rmse(best_lasso_pred, y_validate))

As shown above, the best fitting LASSO model features a tuning parameter value of 0.01. However, the validation R^2 is negative, suggesting that something is wrong with the model. Moreover, the validation RMSE is larger than that of the ridge and OLS models.

In [ ]:
# recreate model, using selected hyperparameters
lasso_reg = Lasso(alpha = 0.01, fit_intercept = True, normalize = True, selection = "cyclic")
lasso_model = lasso_reg.fit(X_train, y_train)

In [ ]:
print(lasso_model.coef_)
print(lasso_model.intercept_)

As shown above and in the table below, all coefficient estimates in this LASSO model have been shrunken to zero. 

In [ ]:
# Create a dataframe with the coefficient and feature names
X_names = diabetes_clean.drop(["Diabetes_Number"], axis=1)
lasso_reg_params = pd.DataFrame([lasso_model.coef_, X_names.columns]).T
lasso_reg_params.columns = ['Coefficient', 'Feature']
lasso_reg_params = lasso_reg_params.convert_dtypes()

In [ ]:
# sort dataframe and show highest 15 absolute value rows
pd.set_option('max_colwidth', 400)
lasso_reg_params_sorted = lasso_reg_params.reindex(lasso_reg_params.Coefficient.abs().sort_values(ascending=False).index)
lasso_sorted = lasso_reg_params_sorted.head(15)
lasso_sorted

In [ ]:
# predict the rate of diabetes
lasso_pred = lasso_model.predict(X_train)
rmse(lasso_pred, y_train)

The training RMSE for this LASSO model is larger than that of the ridge and OLS models.

In [ ]:
# plot actual vs predicted values
plt.scatter(y_train, lasso_pred)
plt.title('Lasso Regression Predicted v. Actual')
plt.xlabel('actual value')
plt.ylabel('predicted value')
plt.show()

As shown in the scatter plot above, since this LASSO model estiamted all coefficients as zero, the predicted value for each observation is simply the intercept, which can is also the mean of the `Diabetes_Number` training data.

## 5. Validate and Refine Models

In [ ]:
# use X_validation and y_validation data sets to evaluate and refine your models

### Model 1: OLS

In [ ]:
# predict the number of riders
lin_pred = lin_model.predict(X_validate)

# plot the residuals on a scatter plot
plt.scatter(y_validate, lin_pred)
plt.title('Linear Model (OLS) Predicted v. Actual')
plt.xlabel('actual value')
plt.ylabel('predicted value')
plt.show()

If this OLS model was 100% accurate, the plot above would have a slope of 1, where every predicted value would match up with every actual value in the set.

Now we can calculate the mean squared error for the model, using the validation data.

In [ ]:
rmse(lin_pred, y_validate)

### Model 2: Ridge

In [ ]:
# predict the rate of diabetes
ridge_pred = ridge_model.predict(X_validate)
rmse(ridge_pred, y_validate)

In [ ]:
# plot actual vs predicted values
plt.scatter(y_validate, ridge_pred)
plt.title('Ridge Predicted v. Actual')
plt.xlabel('actual value')
plt.ylabel('predicted value')
plt.show()

As shown by the RMSE estimate above, the ridge model fits the validation data slightly better than the OLS model.

### Model 3: Lasso

In [ ]:
# predict the rate of diabetes
lasso_pred = lasso_model.predict(X_validate)
rmse(lasso_pred, y_validate)

In [ ]:
# plot actual vs predicted values
plt.scatter(y_validate, lasso_pred)
plt.title('Lasso Predicted v. Actual')
plt.xlabel('actual value')
plt.ylabel('predicted value')
plt.show()

Of the three models considered, the LASSO is the worst fitting model, with an RMSE of 0.028 for the validation data. 

### Feature Selection

In [ ]:
#Feature Selection

# sort dataframe and show highest 10 absolute value rows
pd.set_option('max_colwidth', 400)
ridge_reg_params_sorted = ridge_reg_params.reindex(ridge_reg_params.Coefficient.abs().sort_values(ascending=False).index)
ridge_sorted = ridge_reg_params_sorted.head(10)
ridge_sorted

In [ ]:
list(ridge_sorted.index) #obtain the indices for selected features

In [ ]:
X_train_reduced = X_train[:,list(ridge_sorted.index)]
X_train_reduced.shape #check to make sure the correct number of features are included

In [ ]:
ridge_model = ridge_reg.fit(X_train_reduced, y_train) #fit ridge model on reduced dataset

In [ ]:
print(ridge_model.coef_)
print(ridge_model.intercept_)

In [ ]:
ridge_pred = ridge_model.predict(X_train_reduced)
rmse(ridge_pred, y_train) #calculate RMSE for reduced model

As shown above, the training RMSE for the reduced ridge model is 0.022, which is higher than the RMSE for the full ridge model.

### Testing Data

The best performing model of the 4 examined (OLS, ridge, LASSO, feature-selected ridge), based on validation RMSE, is the full ridge regression model. We will now assess the fit of this model with the testing data.

In [ ]:
# recreate model, using selected hyperparameters
ridge_reg = Ridge(alpha=0.9, fit_intercept= True, normalize= True, solver= 'svd')
ridge_model = ridge_reg.fit(X_train, y_train)

#predicted values using test data 
ridge_pred = ridge_model.predict(X_test)
rmse(ridge_pred, y_test)

In [ ]:
# plot predicted vs. actual for test data
plt.scatter(y_test, ridge_pred)
plt.title('Ridge Predicted v. Actual')
plt.xlabel('actual value')
plt.ylabel('predicted value')
plt.show()

The RMSE for the ridge model as applied to testing data is 0.0203, which is slightly higher than the RMSE from the ridge model applied to the validation data (0.0197), but smaller than the RMSE from the OLS model applied to the validation data (0.0206).

The advantage of using both a validation dataset and a testing dataset is that the models can be tuned and adjusted using both the training and validation datasets, and the true predictive validity of the model can then be examined on the testing dataset, which plays no role in the training of the models.

### Cross-Validation

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

ridge_reg = Ridge(alpha=0.9, fit_intercept= True, normalize= True, solver= 'svd') #best ridge model

cv_size = np.arange(3,11,1) #different cv sizes

for size in cv_size: #iterate over the different sizes and compute the average RMSE
    ridge_pred = cross_val_score(ridge_reg, X, y, cv = size, scoring = "neg_mean_squared_error")
    print(size, ":", np.sqrt(abs(ridge_pred)).mean())

The best fitting cross-validation ridge model uses 6 folds. This model has an average RMSE of 0.238, which isl larger than the RMSE for the ridge model applied to the testing data. 

When considering the number of folds to use in the cross-validation approach, two concerns must be addressed: computational expense and bias-variance tradeoff. As the number of folds increase, the intensity of the computations increase, as a greater number of models are being fit. Regarding the bias-variance tradeoff, as the number of folds increase, bias decreases and variance increases. Thus, while a leave-one-out approach will result in a relatively unbiased estimate of the testing error, the estimate will have higher variance than an approach with a fewer number of folds. 

## 6. Discussion Questions

### 6.1 What is bias-variance tradeoff? Why is it relevant to machine learning problems like this one?

Bias-variance tradeoff is the relationship between how much variance is present in a model and how much bias is present. It is ideal to have both low bias and low variance in a model, but that is often not possible. Bias is the error introduced by using a simpler model to approximate a more complex real-world problem, whereas variance is the amount that the prediction would change had it been estimated using a different training set. 

Bias-variance tradeoff is relevant to machine learning problems like this one because ideally, we want to create a model that is not too simple to yield high bias, but that is also not too complex to become uninterpretable. As the goal of incorporating machine learning into this question is to create new policy, it becomes important that we can draw clear conclusions from a model that is fairly transparent. Additionally, as the policymakers interested in looking at this data are interested in how diabetes rates compare across counties in the United States, it is very important that the data used for training are representative of national data and demographics. Otherwise, the model will have very high variance as predictions will be very sensitive to the training data.

### 6.2 Define overfitting and why it matters for machine learning. How can we address it?

### 6.3 Discuss your analysis in 2-3 paragraphs.